In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Masking
from keras.layers import BatchNormalization
from keras import optimizers
from keras.utils import Sequence
from keras.callbacks import BaseLogger
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors

import matplotlib

import pickle
import pandas as pd
import numpy as np
import math
import itertools
import matplotlib.pyplot as plt
from scipy import spatial

from SequenceHandler import evaluate, flatten, genXY, getSubDict, ranks, splitTrainTest, genXY, gen_sequences
from ShowLoss import ShowLoss

Using TensorFlow backend.


In [2]:
vectors = pickle.load(open("vectors/vectors_doc2vec_sessions.pickle", 'rb'))
flat, xyi = flatten(vectors, seq_len=-1)
traini, testi = splitTrainTest(xyi)
_trainx, _trainy = genXY(flat, traini)
_testx, _testy = genXY(flat, testi)

In [ ]:
_args = {"epochs":50, "steps_per_epoch":10000}
_largs = {"activation":"tanh", "dropout":0}
VECSIZE = 100

nn = Sequential()
nn.add(Masking(mask_value=0., input_shape=(None,VECSIZE)))
nn.add(LSTM(50, return_sequences=True, **_largs))
nn.add(LSTM(50, return_sequences=True, **_largs))
nn.add(LSTM(50, return_sequences=True, **_largs))
nn.add(LSTM(50, return_sequences=True, **_largs))
nn.add(LSTM(50, **_largs))
nn.add(BatchNormalization())
nn.add(Dense(VECSIZE))
nn.compile(loss='cosine_proximity', optimizer="adam")
nn.summary()

nn.fit_generator(generator=gen_sequences(_trainx, _trainy), **_args, verbose=1, callbacks=[ShowLoss()])

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, None, 100)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 50)          30200     
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 50)          20200     
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 50)          20200     
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 50)          20200     
_________________________________________________________________
lstm_5 (LSTM)                (None, 50)                20200     
_________________________________________________________________
batc

In [ ]:
nn.evaluate_generator(generator=gen_sequences(_testx, _testy), steps=2000)

In [ ]:
nn.save("./model/lstm_newarchi")

In [ ]:
nn = load_model("./model/lstm_newarchi")

In [ ]:
evaluate(ranks(flat, traini, lambda x: nn.predict(np.array([x]))[0]))
evaluate(ranks(flat, testi, lambda x: nn.predict(np.array([x]))[0]))